In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn


ModuleNotFoundError: No module named 'torch'

### Data Load

In [ ]:
# 데이터 로드
arr = np.load("../output.npy")
tensor_data = torch.from_numpy(arr)
dataset = TensorDataset(tensor_data, tensor_data)  # Autoencoder는 입력=출력
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


### Model Structure Define

In [ ]:
class ChannelAttention3D(nn.Module):
    def __init__(self, intput_size: int, hidden_size: int = 16):
        super().__init__()
        hidden = max(channels // reduction, 4)
        self.avg_pool = nn.AdaptiveAvgPool3d(1)
        self.max_pool = nn.AdaptiveMaxPool3d(1)
        self.mlp = nn.Sequential(
            nn.
            nn.ReLU(inplace=True),
            nn.Conv3d(hidden, channels, kernel_size=1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.mlp(self.avg_pool(x))
        max_out = self.mlp(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

### Loss, HpyerParameters

In [ ]:
# ----- 모델/손실/옵티마이저 -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder3D().to(device) # -> 여기 MLP로 바꿔야함
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# 시각화용 list
loss_list = []

### Train

In [ ]:
epochs = 300

plt.ion()  # interactive mode ON
fig, ax = plt.subplots(figsize=(8,5))

min_acc = 99999

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(dataloader)

    if avg_loss < min_acc:
        min_acc = avg_loss
        if epoch > 50:
            torch.save(model.state_dict(), f"autoencoder_epoch[{epoch}]_loss[{avg_loss:.3f}].pth")
            
    loss_list.append(avg_loss)

    # --- 실시간 그래프 업데이트 ---
    ax.clear()
    ax.plot(loss_list, label="Training Loss", color="blue")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("MSE Loss")
    ax.set_title("Autoencoder3D Training Loss")
    ax.legend()
    ax.grid(True)
    plt.pause(0.01)  # 짧게 멈추면서 그래프 갱신

    print(f"[Epoch {epoch+1}/{epochs}] Loss: {avg_loss:.6f}")

plt.ioff()  # interactive mode OFF
plt.show()  # 최종 결과 표시

# ----- 모델 저장 -----
torch.save(model.state_dict(), "autoencoder3d.pth")
